In [ ]:
#%%
## IMPORT MODULES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import plot_predict
from statsmodels.tsa.api import ARDL
from statsmodels.tsa.ardl import ardl_select_order
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels as sm
from labellines import labelLines
import matplotx

from model_functions import load_data


In [ ]:
#%%
## SET UP TRAINING AND TEST DATASETS

df_full = load_data()

df_full = df_full[['y', 'rf_mean', 'rf_8h']].rename(columns={'y':'flow'})
# small dataset
'''df_train = df_full.loc['2001'].copy()
df_test = df_full.loc['2002-01': '2002-02'].copy()'''

# large dataset
df_train = df_full.loc['2001':'2014'].copy()
#df_train = df_train[df_train['quality_min'] != 0] # remove missing data
df_val = df_full.loc['2015': '2019'].copy()
df_test = df_full.loc['2020': '2024'].copy()

# remove missing data
df_train = df_train.dropna()
endog = df_train['flow']
rf = df_train['rf_mean']
rf_lag = df_train['rf_8h']


In [ ]:
#%%
#plot ACF and PACF
series_diff = series.diff().dropna()
plot_acf(series_diff); # prevents plotting twice
plot_pacf(series_diff); # 

# best parameters based on acf/pacf: (3,0,6)


In [ ]:
#%%
## ARIMA MODEL

def run_arima(p, q, d=0, exog=None):
    model = ARIMA(endog, exog=exog, order=(p, d, q))
    res = model.fit(method_kwargs={'maxiter':300})
    return res


In [ ]:
#%%
# fit ARIMA model to data
#### takes ~ 20 minutes ####
res = run_arima(11,10, exog=rf_lag)
print(res.aic)
# optimum was (11,0,10), aic = 10196


In [ ]:
#%%
# generate forecast
df_forecast = df_test
steps=len(df_forecast)
forecast = res.forecast(steps, exog=df_forecast[['rf_mean']])
df_forecast['forecast'] = forecast.values # avoids index issues



In [ ]:
#%%
## calculate metrics

# drop null values
forecast_metrics = df_forecast.dropna(subset=['flow','forecast'])
# retrieve true and predicted values for flow
y = forecast_metrics['flow']
y_pred = forecast_metrics['forecast']
# mean squared error
mse = mean_squared_error(y_pred, y)
# Nash-Sutcliffe efficiency (= r^2)
nse = r2_score(y_pred, y)
print('Mean squared error: ', mse, '\nNash-Sutcliffe Efficiency: ', nse)



In [ ]:
#%%
# ARDL MODEL
# (autoregressive distributed lags)
# automatically select best lags for model

#### can take ~ 1 hour ####
rf = pd.DataFrame(rf)
sel_res = ardl_select_order(
    endog, 75, rf, 75, ic="aic", trend="c",
    causal=True, # exclude lag 0 for exog 
    glob=False # search through all 'submodels' (combinations of lags)
)

# for global search, time is of order 2^(maxlag+maxorder) for 1 exogenous variable

print(sel_res.model.ar_lags)
print(sel_res.model.dl_lags)
'''
for i, val in enumerate(sel_res.aic.head(10)):
    print(f"{i+1}: {val}")
'''

# large dataset:
# search up to (75, 75) gives (60, 74)
# aic = 38975


In [ ]:
#%%
# fit ARDL model with specified parameters
rf = pd.DataFrame(rf)
res = ARDL(endog, 60, rf, 74).fit()
res.summary()



In [ ]:
#%%
## generate forecast
df_forecast = df_test
steps=len(df_forecast)
forecast = res.forecast(steps, exog=df_forecast[['rf_mean']])
df_forecast['forecast'] = forecast.values # avoids index issues
#df_forecast['forecast'].plot()
#df_forecast['flow'].plot()


In [ ]:
#%%
## calculate metrics

# drop null values
forecast_metrics = df_forecast.dropna(subset=['flow','forecast'])
# retreive true and predicted values for flow
y = forecast_metrics['flow']
y_pred = forecast_metrics['forecast']
# mean squared error
mse = mean_squared_error(y_pred, y)
# Nash-Sutcliffe efficiency (= r^2)
nse = r2_score(y_pred, y)

print('Mean squared error: ', mse, '\nNash-Sutcliffe Efficiency: ', nse)


In [ ]:
#%%

# plot model predictions
df_forecast = df_forecast.loc['2020-12-25':'2024-12-27']
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(df_forecast.index, df_forecast['flow'], 'blue', 
         label='Observed')
ax1.plot(df_forecast.index, df_forecast['forecast'], 'orange', 
         label='Predicted')
'''ax1.fill_between(df_test.index, 
                 conf_int['lower'], 
                 conf_int['upper'], 
                 color='gray', alpha=0.3, 
                 label='66% confidence interval')'''

# matplotx.line_labels()
plt.legend(loc='best',
           bbox_to_anchor=(0.95,0.85),
            frameon=False,
           #labelcolor='linecolor',
           #handlelength=0
           )
plt.gca().set_ylim(bottom=0)

# add rainfall 
# Creating a secondary y-axis for the upside-down histogram
ax2 = ax1.twinx()
# Plot the histogram
df_plot_day = df_forecast.resample('3h').sum()
ax2.bar(df_plot_day.index,
            df_plot_day['rf_mean'],
            width=(1/8),
            #align='center'
            )

# Invert the y-axis to make the histogram upside-down
#ax2.invert_yaxis()
# Optional: Adjust the y-axis limit to match the main plot for a consistent look
#ax_top = ax1.get_ylim()[1]
ax2.set_ylim(ax1.get_ylim()[::-1])

#plt.xlim(df_test_day.index[0], df_test_day.index[1])

#ax1.set_xlabel('Date')
ax1.set_ylabel('Flow (m$^3$/s)')
ax2.set_ylabel('Rainfall (mm)', color='b')
ax2.tick_params(axis='y', labelcolor='b')
ax1.spines[:].set_visible(False)
ax2.spines[:].set_visible(False)


